In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 16.7 MB/s eta 0:00:00


In [10]:
import json
import os
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, pipeline
import torch
from sentence_transformers import SentenceTransformer, models
import requests
from google.colab import files
import time
import re
from collections import defaultdict

# Load Together API key manually for Colab
TOGETHER_API_KEY = "1fe0076c178dc228a74675178f812ae53c34207ae9a271bb9c370530889cfaed"

# Test questions
test_questions = [
    "What medication is recommended for acute bipolar depression?",
    "What are alternative treatments for a patient who had side effects from quetiapine?",
    "What is the recommended treatment for mixed episodes in bipolar disorder?",
    "What is the first-line treatment for Bipolar II disorder acute depression?",
    "What medication should be avoided in obese patients with bipolar disorder?",
    "What medication considerations are there for pregnant patients with bipolar disorder?",
    "What medication should be used cautiously in patients with diabetes?",
    "What medication considerations are there for patients with hepatic impairment?",
    "What medication should be used cautiously in patients with cardiovascular disease?",
    "What medication considerations are there for patients with renal impairment?",
    "How should rapid cycling bipolar disorder be treated?",
    "What is the recommended treatment for schizoaffective disorder?",
    "What is the role of lamotrigine as adjunct therapy?",
    "When is electroconvulsive therapy recommended?",
    "What treatments show rapid response within the first week?",
    "What medications help with comorbid anxiety in bipolar disorder?",
    "What is recommended for lithium maintenance therapy?",
    "What is recommended for quetiapine maintenance therapy?",
    "What is recommended for Bipolar II maintenance therapy?",
    "What are common gastrointestinal side effects of mood stabilizers?"
]

# Enhanced data loading with better structure preservation
def load_json_knowledge():
    uploaded = files.upload()
    if not uploaded:
        raise RuntimeError("❌ No file uploaded.")
    json_filename = next(iter(uploaded))
    with open(json_filename, 'r', encoding='utf-8') as f:
        data = json.load(f)

    entries = []
    metadata = []

    def extract_content(obj, path=""):
        if isinstance(obj, dict):
            # Look for text content in various fields
            text_fields = ['text', 'content', 'description', 'recommendation', 'guideline']
            found_text = None

            for field in text_fields:
                if field in obj and obj[field]:
                    found_text = str(obj[field]).strip()
                    break

            if found_text:
                entries.append(found_text)
                metadata.append({
                    'path': path,
                    'source': obj.get('source', ''),
                    'section': obj.get('section', ''),
                    'type': obj.get('type', ''),
                    'medication': extract_medications(found_text)
                })

            # Recursively process nested objects
            for key, value in obj.items():
                if key not in text_fields:
                    extract_content(value, f"{path}.{key}" if path else key)

        elif isinstance(obj, list):
            for i, item in enumerate(obj):
                extract_content(item, f"{path}[{i}]" if path else f"[{i}]")
        else:
            text = str(obj).strip()
            if len(text) > 50:  # Only include substantial text
                entries.append(text)
                metadata.append({
                    'path': path,
                    'medication': extract_medications(text)
                })

    extract_content(data)

    # Clean entries
    cleaned_entries = []
    cleaned_metadata = []
    for entry, meta in zip(entries, metadata):
        cleaned = re.sub(r'\s+', ' ', entry).strip()
        if len(cleaned) > 20:
            cleaned_entries.append(cleaned)
            cleaned_metadata.append(meta)

    print(f"📚 Loaded {len(cleaned_entries)} knowledge entries with metadata")
    return cleaned_entries, cleaned_metadata

def extract_medications(text):
    """Extract medication names from text"""
    medications = set()
    text_lower = text.lower()

    # Common bipolar medications to look for
    bipolar_meds = [
        'quetiapine', 'seroquel', 'lithium', 'lamotrigine', 'lamictal',
        'divalproex', 'depakote', 'valproate', 'carbamazepine', 'tegretol',
        'lurasidone', 'latuda', 'aripiprazole', 'abilify', 'olanzapine',
        'zyprexa', 'risperidone', 'risperdal', 'cariprazine', 'vraylar',
        'ziprasidone', 'geodon', 'haloperidol', 'haldol', 'fluoxetine',
        'prozac', 'sertraline', 'zoloft', 'paroxetine', 'paxil'
    ]

    for med in bipolar_meds:
        if med in text_lower:
            medications.add(med)

    return list(medications)

# Enhanced embedder with medical focus
class MedicalEmbedder:
    def __init__(self):
        try:
            # Try medical-specific models in order of preference
            models_to_try = [
                'emilyalsentzer/Bio_ClinicalBERT',
                'pritamdeka/S-PubMedBert-MS-MARCO',
                'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext',
                'all-MiniLM-L6-v2'
            ]

            for model_name in models_to_try:
                try:
                    if 'Bio_ClinicalBERT' in model_name:
                        word_embedding_model = models.Transformer(model_name)
                        pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
                        self.model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
                    else:
                        self.model = SentenceTransformer(model_name)
                    print(f"✅ Using {model_name}")
                    break
                except Exception as e:
                    print(f"⚠️ Failed to load {model_name}: {e}")
                    continue
        except Exception as e:
            print(f"❌ Error initializing embedder: {e}")
            raise

    def encode(self, texts, show_progress_bar=True):
        return self.model.encode(texts, show_progress_bar=show_progress_bar, batch_size=16)

# Enhanced search with multiple strategies
class EnhancedSearch:
    def __init__(self, passages, metadata, embedder):
        self.passages = passages
        self.metadata = metadata
        self.embedder = embedder
        self.build_indices()

    def build_indices(self):
        print("🔗 Building search indices...")

        # Main semantic index
        embeddings = self.embedder.encode(self.passages)
        self.semantic_index = faiss.IndexHNSWFlat(embeddings.shape[1], 32)
        self.semantic_index.hnsw.efConstruction = 40
        self.semantic_index.add(embeddings.astype('float32'))

        # Medication-specific index
        self.med_passages = []
        self.med_indices = []
        for i, (passage, meta) in enumerate(zip(self.passages, self.metadata)):
            if meta.get('medication') or self.contains_medication_terms(passage):
                self.med_passages.append(passage)
                self.med_indices.append(i)

        if self.med_passages:
            med_embeddings = self.embedder.encode(self.med_passages)
            self.med_index = faiss.IndexHNSWFlat(med_embeddings.shape[1], 32)
            self.med_index.add(med_embeddings.astype('float32'))
        else:
            self.med_index = None

        print(f"📊 Built semantic index: {len(self.passages)} passages")
        print(f"💊 Built medication index: {len(self.med_passages)} passages")

    def contains_medication_terms(self, text):
        """Check if text contains medication-related terms"""
        med_keywords = [
            'medication', 'drug', 'treatment', 'therapy', 'prescription',
            'dose', 'dosing', 'mg', 'side effect', 'adverse', 'contraindic',
            'first-line', 'second-line', 'maintenance', 'acute', 'recommended'
        ]
        text_lower = text.lower()
        return any(keyword in text_lower for keyword in med_keywords)

    def search(self, query, top_k=10):
        results = []

        # Strategy 1: Semantic search on all passages
        semantic_results = self._semantic_search(query, top_k)
        results.extend(semantic_results)

        # Strategy 2: Medication-focused search if query is about medications
        if self._is_medication_query(query) and self.med_index is not None:
            med_results = self._medication_search(query, top_k//2)
            results.extend(med_results)

        # Strategy 3: Keyword-based search for specific terms
        keyword_results = self._keyword_search(query, top_k//2)
        results.extend(keyword_results)

        # Deduplicate and rank by relevance
        seen_indices = set()
        unique_results = []
        for result in results:
            if result['index'] not in seen_indices:
                unique_results.append(result)
                seen_indices.add(result['index'])

        # Sort by score and return top results
        unique_results.sort(key=lambda x: x['score'])
        return unique_results[:top_k]

    def _semantic_search(self, query, top_k):
        query_embedding = self.embedder.encode([query])
        self.semantic_index.hnsw.efSearch = 64
        D, I = self.semantic_index.search(query_embedding.astype('float32'), top_k)

        results = []
        for score, idx in zip(D[0], I[0]):
            if idx < len(self.passages):
                results.append({
                    'text': self.passages[idx],
                    'score': float(score),
                    'index': int(idx),
                    'method': 'semantic'
                })
        return results

    def _medication_search(self, query, top_k):
        if not self.med_index:
            return []

        query_embedding = self.embedder.encode([query])
        D, I = self.med_index.search(query_embedding.astype('float32'), top_k)

        results = []
        for score, idx in zip(D[0], I[0]):
            if idx < len(self.med_passages):
                original_idx = self.med_indices[idx]
                results.append({
                    'text': self.med_passages[idx],
                    'score': float(score) * 0.9,  # Slight boost for medication-specific results
                    'index': original_idx,
                    'method': 'medication'
                })
        return results

    def _keyword_search(self, query, top_k):
        """Simple keyword-based search for backup"""
        query_terms = query.lower().split()
        results = []

        for i, passage in enumerate(self.passages):
            passage_lower = passage.lower()
            matches = sum(1 for term in query_terms if term in passage_lower)
            if matches > 0:
                # Simple scoring based on term frequency
                score = -matches / len(query_terms)  # Negative because lower is better in our scoring
                results.append({
                    'text': passage,
                    'score': score,
                    'index': i,
                    'method': 'keyword'
                })

        results.sort(key=lambda x: x['score'])
        return results[:top_k]

    def _is_medication_query(self, query):
        med_indicators = [
            'medication', 'drug', 'treatment', 'therapy', 'prescription',
            'recommended', 'first-line', 'alternative', 'side effect',
            'quetiapine', 'lithium', 'lamotrigine', 'lurasidone', 'divalproex'
        ]
        query_lower = query.lower()
        return any(indicator in query_lower for indicator in med_indicators)

# Improved answer generation with context analysis
def generate_answer_with_analysis(query, search_results, max_context_length=3000):
    """Generate answer with better context selection and analysis"""

    # Analyze context to prioritize relevant chunks
    analyzed_chunks = []
    for result in search_results[:8]:  # Analyze top 8 results
        chunk = result['text']
        relevance_score = calculate_relevance(query, chunk)
        analyzed_chunks.append({
            'text': chunk,
            'relevance': relevance_score,
            'method': result.get('method', 'unknown')
        })

    # Sort by relevance and select best chunks within token limit
    analyzed_chunks.sort(key=lambda x: x['relevance'], reverse=True)

    selected_chunks = []
    current_length = 0
    for chunk_info in analyzed_chunks:
        chunk_text = chunk_info['text']
        if current_length + len(chunk_text) <= max_context_length:
            selected_chunks.append(chunk_text)
            current_length += len(chunk_text)
        else:
            # Try to fit a truncated version
            remaining_space = max_context_length - current_length
            if remaining_space > 200:  # Only if we have meaningful space left
                selected_chunks.append(chunk_text[:remaining_space-3] + "...")
            break

    context = "\n\n".join(selected_chunks)

    # Create more targeted prompt based on query type
    query_type = classify_query_type(query)
    prompt = create_targeted_prompt(query, context, query_type)

    return call_api_with_fallback(prompt)

def calculate_relevance(query, chunk):
    """Calculate relevance score between query and chunk"""
    query_lower = query.lower()
    chunk_lower = chunk.lower()

    score = 0

    # Exact keyword matches
    query_words = set(query_lower.split())
    chunk_words = set(chunk_lower.split())
    common_words = query_words & chunk_words
    score += len(common_words) * 2

    # Medication name matches (higher weight)
    bipolar_meds = [
        'quetiapine', 'lithium', 'lamotrigine', 'divalproex', 'lurasidone',
        'aripiprazole', 'olanzapine', 'cariprazine', 'carbamazepine'
    ]
    for med in bipolar_meds:
        if med in query_lower and med in chunk_lower:
            score += 10

    # Clinical terms
    clinical_terms = [
        'bipolar', 'depression', 'mania', 'maintenance', 'acute',
        'first-line', 'second-line', 'recommended', 'contraindic'
    ]
    for term in clinical_terms:
        if term in query_lower and term in chunk_lower:
            score += 3

    # Length bonus for substantial content
    if len(chunk) > 500:
        score += 1

    return score

def classify_query_type(query):
    """Classify the type of clinical query"""
    query_lower = query.lower()

    if any(term in query_lower for term in ['acute', 'depression', 'depressive']):
        return 'acute_treatment'
    elif any(term in query_lower for term in ['maintenance', 'long-term', 'prophylaxis']):
        return 'maintenance'
    elif any(term in query_lower for term in ['side effect', 'adverse', 'contraindic']):
        return 'safety'
    elif any(term in query_lower for term in ['alternative', 'different', 'switch']):
        return 'alternative'
    elif any(term in query_lower for term in ['first-line', 'recommended', 'treatment']):
        return 'recommendation'
    else:
        return 'general'

def create_targeted_prompt(query, context, query_type):
    """Create a targeted prompt based on query type"""

    base_instruction = """You are a clinical decision support system for bipolar disorder treatment. Provide specific, evidence-based recommendations using ONLY the clinical guidelines provided."""

    type_specific_instructions = {
        'acute_treatment': "Focus on first-line and second-line acute treatment options. Prioritize medications specifically mentioned for acute episodes.",
        'maintenance': "Focus on maintenance therapy recommendations and long-term treatment strategies.",
        'safety': "Focus on contraindications, side effects, and safety considerations for specific populations.",
        'alternative': "Focus on alternative treatment options when first-line treatments fail or cause side effects.",
        'recommendation': "Focus on evidence-based treatment recommendations and clinical guidelines.",
        'general': "Provide comprehensive treatment recommendations based on the available evidence."
    }

    specific_instruction = type_specific_instructions.get(query_type, type_specific_instructions['general'])

    prompt = f"""{base_instruction}

{specific_instruction}

IMPORTANT GUIDELINES:
- Use EXACT medication names from the context
- Specify treatment line (first-line, second-line, etc.) when mentioned
- Include relevant clinical details (dosing, monitoring, contraindications)
- If multiple options exist, list them clearly
- Base recommendations STRICTLY on the provided context
- Do not recommend lamotrigine unless it's specifically indicated in the context for this exact clinical scenario

CLINICAL CONTEXT:
{context}

CLINICAL QUESTION: {query}

EVIDENCE-BASED RECOMMENDATION:"""

    return prompt

def call_api_with_fallback(prompt):
    """Call API with multiple model fallbacks"""
    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }

    models = [
        "mistralai/Mistral-7B-Instruct-v0.1",
        "meta-llama/Llama-2-7b-chat-hf"
    ]

    for model in models:
        try:
            payload = {
                "model": model,
                "prompt": prompt,
                "max_tokens": 400,
                "temperature": 0.05,  # Very low temperature for consistency
                "top_p": 0.85,
                "stop": ["CLINICAL QUESTION:", "CLINICAL CONTEXT:"]
            }

            response = requests.post("https://api.together.xyz/v1/completions", json=payload, headers=headers)
            result = response.json()

            if "choices" in result and result["choices"]:
                answer = result["choices"][0]["text"].strip()
                # Clean up response
                answer = re.sub(r'^(Based on.*?:)', '', answer).strip()
                answer = re.sub(r'^(EVIDENCE-BASED RECOMMENDATION:)', '', answer).strip()
                return answer

        except Exception as e:
            print(f"⚠️ Error with {model}: {e}")
            continue

    return "Unable to generate recommendation due to API issues."

# Golden answers (keeping original structure)
question_keywords = [
    ["quetiapine"],
    ["lurasidone", "lithium", "divalproex"],
    ["divalproex", "atypical antipsychotic"],
    ["quetiapine"],
    ["lithium"],
    ["lithium", "pregnant"],
    ["lithium", "diabetes"],
    ["lithium", "hepatic"],
    ["quetiapine", "cardiovascular"],
    ["quetiapine", "renal"],
    ["mania", "rapid cycling"],
    ["schizoaffective", "antipsychotic", "mood stabilizer"],
    ["lamotrigine", "adjunct"],
    ["electroconvulsive"],
    ["electroconvulsive", "cariprazine", "olanzapine-fluoxetine"],
    ["quetiapine", "lurasidone", "anxiety"],
    ["lithium", "maintenance"],
    ["quetiapine", "maintenance"],
    ["quetiapine", "bdii", "maintenance"],
    ["nausea", "vomiting", "diarrhea"]
]

question_incorrect_keywords = [
    ["lamotrigine"],
    ["aripiprazole", "lamotrigine"],
    [],
    ["lamotrigine"],
    ["lamotrigine"],
    [],
    [],
    [],
    [],
    ["lithium"],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    []
]

question_comments = [
    "",
    "Vivomap doesn't specify the combo; lists meds only",
    "CANMAT recommends divalproex and atypical antipsychotics",
    "",
    "Vivomap flags obesity but not weight gain",
    "",
    "",
    "",
    "",
    "Use quetiapine but caution in renal disease",
    "Address rapid cycling risk factors",
    "Use atypical antipsychotic or combo with mood stabilizer",
    "Adjunct lamotrigine is first-line",
    "ECT is first-line for rapid response",
    "ECT and cariprazine/olanzapine-fluoxetine show week-1 effects",
    "Quetiapine and lurasidone help with anxiety",
    "Continue lithium for maintenance",
    "Continue quetiapine for maintenance",
    "Continue quetiapine for BDII maintenance",
    "Common: nausea, vomiting, diarrhea"
]

# Main evaluation function
def run_evaluation(passages, metadata, search_engine):
    print("\n📊 ENHANCED EVALUATION SYSTEM")
    print("="*50)

    correct_count = 0
    total_questions = min(len(test_questions), len(question_keywords))

    for i in range(total_questions):
        query = test_questions[i]
        print(f"\n🔍 Q{i+1}: {query}")

        try:
            # Enhanced search
            search_results = search_engine.search(query, top_k=10)

            # Generate answer with analysis
            answer = generate_answer_with_analysis(query, search_results)

            # Evaluation
            prediction = answer.lower()
            gold_keywords = [kw.lower() for kw in question_keywords[i]]
            incorrect_keywords = [kw.lower() for kw in question_incorrect_keywords[i]]

            has_correct = any(kw in prediction for kw in gold_keywords) if gold_keywords else True
            has_incorrect = any(kw in prediction for kw in incorrect_keywords) if incorrect_keywords else False

            correct = has_correct and not has_incorrect

            if correct:
                correct_count += 1

            symbol = "✅" if correct else "❌"

            print(f"{symbol} Q{i+1}:")
            print(f"    🔹Generated: {answer[:150]}{'...' if len(answer) > 150 else ''}")
            print(f"    🎯Expected: {', '.join(question_keywords[i])}")
            if question_incorrect_keywords[i]:
                print(f"    🚫Avoid: {', '.join(question_incorrect_keywords[i])}")
            if question_comments[i]:
                print(f"    💬Note: {question_comments[i]}")
            print(f"    ✅ Result: {'CORRECT' if correct else 'INCORRECT'}")

            # Show search method breakdown
            methods = [r.get('method', 'unknown') for r in search_results[:3]]
            print(f"    🔍 Search methods: {', '.join(methods)}")

            time.sleep(1.1)  # Rate limiting

        except Exception as e:
            print(f"❌ Error in Q{i+1}: {e}")

    accuracy = (correct_count / total_questions) * 100
    print(f"\n🎯 FINAL RESULTS:")
    print(f"✅ Correct: {correct_count}/{total_questions}")
    print(f"📊 Accuracy: {accuracy:.1f}%")

    # Interactive mode
    print(f"\n{'='*50}")
    print("🩺 Interactive Clinical Q&A")
    print(f"{'='*50}")

    while True:
        user_query = input("\n💬 Enter clinical question (or 'exit'): ")
        if user_query.lower() == 'exit':
            break

        try:
            search_results = search_engine.search(user_query, top_k=8)
            answer = generate_answer_with_analysis(user_query, search_results)
            print(f"\n🩺 Clinical Recommendation:\n{answer}")

            # Show evidence sources
            print(f"\n📚 Based on {len(search_results)} evidence sources:")
            for i, result in enumerate(search_results[:3]):
                print(f"  {i+1}. {result['method']} search (score: {result['score']:.3f})")

        except Exception as e:
            print(f"❌ Error: {e}")

# Main execution
if __name__ == "__main__":
    print("🏥 ENHANCED BIPOLAR DISORDER CLINICAL DECISION SUPPORT")
    print("="*60)

    print("\n📁 Upload your clinical guidelines JSON file...")
    passages, metadata = load_json_knowledge()

    print("\n🔬 Initializing medical embedder...")
    embedder = MedicalEmbedder()

    print("\n🔍 Building enhanced search system...")
    search_engine = EnhancedSearch(passages, metadata, embedder)

    print("\n🧪 Running enhanced evaluation...")
    run_evaluation(passages, metadata, search_engine)

🏥 ENHANCED BIPOLAR DISORDER CLINICAL DECISION SUPPORT

📁 Upload your clinical guidelines JSON file...


Saving guideline_db.json to guideline_db (3).json
📚 Loaded 718 knowledge entries with metadata

🔬 Initializing medical embedder...
✅ Using emilyalsentzer/Bio_ClinicalBERT

🔍 Building enhanced search system...
🔗 Building search indices...


Batches:   0%|          | 0/45 [00:00<?, ?it/s]

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

📊 Built semantic index: 718 passages
💊 Built medication index: 368 passages

🧪 Running enhanced evaluation...

📊 ENHANCED EVALUATION SYSTEM

🔍 Q1: What medication is recommended for acute bipolar depression?


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Q19:
    🔹Generated: For Bipolar II maintenance therapy, the 2016 CANMAT guidelines recommend a combination of antidepressants and psychotherapy (Level 1). Specifically, f...
    🎯Expected: quetiapine, bdii, maintenance
    💬Note: Continue quetiapine for BDII maintenance
    ✅ Result: CORRECT
    🔍 Search methods: keyword, keyword, keyword

🔍 Q20: What are common gastrointestinal side effects of mood stabilizers?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Q20:
    🔹Generated: 1. Mood stabilizers, including lithium, valproic acid, carbamazepine, and levetiracetam, can cause gastrointestinal side effects such as nausea, vomit...
    🎯Expected: nausea, vomiting, diarrhea
    💬Note: Common: nausea, vomiting, diarrhea
    ✅ Result: CORRECT
    🔍 Search methods: keyword, keyword, keyword

🎯 FINAL RESULTS:
✅ Correct: 16/20
📊 Accuracy: 80.0%

🩺 Interactive Clinical Q&A

💬 Enter clinical question (or 'exit'): exit
